In [1]:
# Python program to scrape table from website
  
# import libraries selenium and time
from selenium import webdriver
from time import sleep
  
# Create webdriver object
driver = webdriver.Chrome(
    executable_path="C:\selenium\chromedriver_win32\chromedriver.exe")
  
# Get the website
driver.get(
    "https://www.geeksforgeeks.org/find_element_by_link_text-driver-method-selenium-python/")
  
# Make Python sleep for some time
sleep(2)
  
# Obtain the number of rows in body
rows = 1+len(driver.find_elements_by_xpath(
    "/html/body/div[3]/div[2]/div/div[1]/div/div/div/article/div[3]/div/table/tbody/tr"))
  
# Obtain the number of columns in table
cols = len(driver.find_elements_by_xpath(
    "/html/body/div[3]/div[2]/div/div[1]/div/div/div/article/div[3]/div/table/tbody/tr[1]/td"))
  
# Print rows and columns
print(rows)
print(cols)
  
# Printing the table headers
print("Locators           "+"             Description")
  
# Printing the data of the table
for r in range(2, rows+1):
    for p in range(1, cols+1):
        
        # obtaining the text from each column of the table
        value = driver.find_element_by_xpath(
            "/html/body/div[3]/div[2]/div/div[1]/div/div/div/article/div[3]/div/table/tbody/tr["+str(r)+"]/td["+str(p)+"]").text
        print(value, end='       ')
    print()

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [2]:
import requests
import pandas as pd

previous_games = 10

pp = {'Player Name':['Donovan Mitchell', 'Kawhi Leonard', 'Rudy Gobert', 'Paul George','Reggie Jackson', 'Jordan Clarkson'],
      'Fantasy Score': [46.0, 50.0, 40.0, 44.0, 25.0, 26.5]}


#Creating a dataframe from dictionary
dfNBA = pd.DataFrame(pp)

search_api = 'https://site.api.espn.com/apis/search/v2'
for idx, row in dfNBA.iterrows():
    playerName = row['Player Name']
    payload = {'query': '%s' %playerName}

    results = requests.get(search_api, params=payload).json()['results']
    for each in results:
        if each['type'] == 'player':
            playerID = each['contents'][0]['uid'].split('a:')[-1]
            break
        
    player_api = 'https://site.web.api.espn.com/apis/common/v3/sports/basketball/nba/athletes/%s/gamelog' %playerID
    playload = {'season':'2021' }
    jsonData_player = requests.get(player_api, params=payload).json()
    
    #Scraping data from last x games
    last_x_gameIDs = list(jsonData_player['events'].keys())
    last_x_gameIDs.sort()
    last_x_gameIDs = last_x_gameIDs[-1*previous_games:]
    
    gamelog_dict = {}
    seasonTypes = jsonData_player['seasonTypes']
    for gameID in last_x_gameIDs:
        for each in seasonTypes:
            categories = each['categories']
            for category in categories:
                if category['type'] == 'total':
                    continue
                events = category['events']
                for event in events:
                    if gameID == event['eventId']:
                        gamelog_dict[gameID] = event['stats']
                    

    labels = jsonData_player['labels']
    
    # Aggrigate totals
    for k, v in gamelog_dict.items():
        v = dict(zip(labels, v))
        gamelog_dict[k] = v
        
    stats = pd.DataFrame(gamelog_dict.values())
    
    points = stats['PTS'].astype(float).sum() / previous_games
    rebs = stats['REB'].astype(float).sum() / previous_games
    asts = stats['AST'].astype(float).sum() / previous_games
    blks = stats['BLK'].astype(float).sum() / previous_games
    stls = stats['STL'].astype(float).sum() / previous_games
    tnvrs = stats['TO'].astype(float).sum() /previous_games

    projectedPoints = float(points)+(float(rebs)*1.2)+(float(asts)*1.5)+(float(blks)*3)+(float(stls)*3)-(float(tnvrs)*1)
    print('%s: %.02f' %(playerName,projectedPoints))

Donovan Mitchell: 41.32
Kawhi Leonard: 8.74
Rudy Gobert: 33.93
Paul George: 37.62
Reggie Jackson: 17.33
Jordan Clarkson: 27.42


In [11]:
import requests
from bs4 import BeautifulSoup

def data():
    League = ['nba', 'nfl', 'mlb']
    url = f"http://www.espn.com/{League[0]}/schedule"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    for row in soup.select("table.schedule tbody tr"):
        home_team, away_team = row.select(".team-name")

        print(home_team.get_text(), away_team.get_text())

    date = soup.find('h2',{'class':'table-caption'})
    #return date.string

In [12]:
result = data()
result

In [14]:
# The following script scrapes ESPN's MLB Standings Grid and writes the
# standings for each American League (AL) team to a CSV file, which has the following
# format:
# Team, Opponent, Wins, Losses

from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import csv

csv_filename = 'AL-standings.csv'

year = '2013'
url = 'http://espn.go.com/mlb/standings/grid/_/year/' + year

page = urlopen(url)
soup = BeautifulSoup(page.read())

# Extracts the table for the American League (AL) and the rows for each team
AL_table = soup.find(text = re.compile("American")).find_parent("table")
AL_rows = AL_table.findAll('tr', class_ = re.compile("team"))

# Creates a list of the AL teams and then appends NL for National League
AL_teams = [team_row.find('b').text for team_row in AL_rows]
AL_teams.append("NL")

# Opens a CSV file for the AL standings
with open(csv_filename, 'wb') as f:
    csv_out = csv.writer(f)
    csv_out.writerow(['Team', 'Opponent', 'Wins', 'Losses'])
    
    # For each team in the AL table, identifies the team's name, the opponent,
    # and their wins and losses (WL) against that opponent. Then outputs the
    # results to the open CSV file
    for team_row in AL_rows:
        team = team_row.find('b').text
        
        # A cell has the following form:
        # <td align="right">
        # 7-9</td>
        WL_cells = team_row.findAll('td', align = "right")
        
        # Extracts the values for both wins and losses from each WL table cell
        wins_losses = [td_cell.text.strip('\n').split('-') for td_cell in WL_cells]
        
        # Writes the current team's standings to the CSV file
        for i, opponent in enumerate(AL_teams):
            if team != opponent:
                csv_out.writerow([team, opponent, wins_losses[i][0], wins_losses[i][1]])

TypeError: a bytes-like object is required, not 'str'

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re


class App:
	def __init__(self,player_name=input("Enter the player name you want to search ")):
		self.player_name=player_name
		if self.player_name == '':
			print("Player name not given")
			exit()
		self.driver=webdriver.Chrome('/usr/local/bin/chromedriver')
		self.url=self.search_profile()
		if not self.url is None:
			personal_data,bowling_record,batting_record=self.parse_profile()
			print ("Personal Data\n")
			print (personal_data)
			print ("\nBatting Record\n")
			print (batting_record)
			print ("\nBowling Record\n")
			print (bowling_record)
		else:
			print("Player profile not found")
			exit()
		self.driver.close()
		
	def search_profile(self):
		
		profile_found=False
		url=None
		self.driver.get('https://www.google.com')
		try:
			input_box = WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH, '//input[@title="Search"]')));
		except TimeoutException:
			print("Connection timeout during googling")
			self.driver.close()
			exit()
		input_box.send_keys(self.player_name)
		input_box.send_keys(Keys.ENTER)
		try:
			#results = WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH, '//div[@class="med"]'))).find_elements_by_tag_name('a')
                        results = self.driver.find_elements_by_xpath("//div[@class='r']/a[@href]")
		except TimeoutException:
			print("Connection timeout fetching results")
			self.driver.close()
			exit()
		for result in results:
			espn=re.search('^http://www.espncricinfo.com', result.get_attribute("href"))
			if espn and 'player' in result.get_attribute("href"):
				profile_found=True
				url=result.get_attribute("href")
				break
		return url
	
	def parse_profile(self):
		data={}
		bowling_record={}
		batting_record={}
		self.driver.get(self.url)
		urls=self.driver.find_elements_by_class_name('ciPlayerinformationtxt')
		for url in urls:
			b=url.find_element_by_tag_name('b')
			span=url.find_element_by_tag_name('span')
			data[b.text]=span.text
			personal_data=dict((key,value) for key, value in data.items() if key in ['Playing role','Major teams','Current age','Height','Born','Batting style','Bowling style','Full name'] )
		data_rows=self.driver.find_elements_by_xpath("//tr[@class='data1']")
		
		
		if len(data_rows)<20:
			format_played=4
		elif len(data_rows) < 28:
			format_played=5
		else:
			format_played=6
			
		batting_rows=data_rows[0:format_played]
		
		for row in batting_rows:
			columns=row.find_elements_by_tag_name('td')
			data_format={}
			data_format['Matches']=columns[1].text
			data_format['Innings']=columns[2].text
			data_format['Not Outs']=columns[3].text
			data_format['Runs']=columns[4].text
			data_format['Highest Score']=columns[5].text
			data_format['Average']=columns[6].text
			data_format['Balls Faced']=columns[7].text
			data_format['Strike Rate']=columns[8].text
			data_format['Centuries']=columns[9].text
			data_format['Half Centuries']=columns[10].text
			data_format['Boundries']=columns[11].text
			data_format['Sixers']=columns[12].text
			data_format['Catches']=columns[13].text
			data_format['Stumpings']=columns[14].text
			batting_record[columns[0].text]=data_format
				
		bowling_rows=self.driver.find_elements_by_xpath("//tr[@class='data1']")[format_played:format_played*2]
		for row in bowling_rows:
			columns=row.find_elements_by_tag_name('td')
			data_format={}
			data_format['Matches']=columns[1].text
			data_format['Innings']=columns[2].text
			data_format['Balls']=columns[3].text
			data_format['Runs']=columns[4].text
			data_format['Wickets']=columns[5].text
			data_format['BBI']=columns[6].text
			data_format['BBM']=columns[7].text
			data_format['Average']=columns[8].text
			data_format['Economy']=columns[9].text
			data_format['Strike Rate']=columns[10].text
			data_format['4 Wickets']=columns[11].text
			data_format['5 Wickets']=columns[12].text
			data_format['10 Wickets']=columns[13].text
			bowling_record[columns[0].text]=data_format
		return personal_data,bowling_record,batting_record
			
		
if __name__=='__main__':
	app=App()
	


	

KeyboardInterrupt: Interrupted by user

In [59]:
months = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:30}
for month in months.keys():
    maxDayNum = int(months.get(month))
    for day in range(1,maxDayNum):

31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
28
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [65]:
chromedriver_autoinstaller.install()

options = Options()
options.headless = True
months = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:30}
for month in months.keys():
    maxDayNum = int(months.get(month))
    if month < 10:
        month = "0"+str(month)
    for day in range(1,maxDayNum):
        if day < 10:
            day = "0"+str(day)
        print("month:" + str(month) + "day: " + str(day))
        nba_schedule = 'https://www.espn.com/nba/schedule/_/date/2022{}{}'.format(month,day)
        driver = webdriver.Chrome(options = options)
        driver.get(nba_schedule)
        nba_table = driver.find_element(By.CLASS_NAME, 'Table') #Prints first table from NFL schedule
        result = nba_table.text
        print(result)
driver.close()

month:01day: 01
MATCHUP
RESULT
WINNER HIGH
LOSER HIGH
New Orleans
  @  
Milwaukee
MIL 136, NO 113
Giannis Antetokounmpo35 Pts
Jaxson Hayes23 Pts
San Antonio
  @  
Detroit
DET 117, SA 116 (OT)
Hamidou Diallo34 Pts
Bryn Forbes27 Pts
Chicago
  @  
Washington
CHI 120, WSH 119
Zach LaVine35 Pts
Kyle Kuzma29 Pts
LA
  @  
Brooklyn
LAC 120, BKN 116
Eric Bledsoe27 Pts
James Harden34 Pts
Denver
  @  
Houston
DEN 124, HOU 111
Nikola Jokic24 Pts
Jalen Green29 Pts
Golden State
  @  
Utah
GS 123, UTAH 116
Stephen Curry28 Pts
Rudy Gobert20 Pts
month:01day: 02
MATCHUP
RESULT
WINNER HIGH
LOSER HIGH
New York
  @  
Toronto
TOR 120, NY 105
Fred VanVleet35 Pts
Evan Fournier20 Pts
Orlando
  @  
Boston
BOS 116, ORL 111 (OT)
Jaylen Brown50 Pts
Terrence Ross33 Pts
Indiana
  @  
Cleveland
CLE 108, IND 104
Evan Mobley24 Pts
Domantas Sabonis32 Pts
Miami
  @  
Sacramento
SAC 115, MIA 113
Buddy Hield26 Pts
Omer Yurtseven22 Pts
Phoenix
  @  
Charlotte
PHX 133, CHA 99
Devin Booker24 Pts
LaMelo Ball17 Pts
Dallas
  @  

KeyboardInterrupt: 

In [69]:
from bs4 import BeautifulSoup

In [70]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

options = Options()
options.headless = True


nba_schedule = 'https://www.espn.com/nba/schedule/_/date/20221110'
driver = webdriver.Chrome(options = options)
driver.get(nba_schedule)

html_str = str(driver.page_source)
page_soup = BeautifulSoup(html_str, 'html.parser')
    

#Prints first table from NFL schedule
# result = nba_table.text
# print(result)
driver.close()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [72]:
nba_table = page_soup.find_all("span", {"class": "Table__Team"}) 

for x in nba_table:
    print(x)
    print('-----')


<span class="Table__Team away"><a class="AnchorLink" data-track-linkid="nba:schedule:team" data-track-navmethod="schedule" href="/nba/team/_/name/dal/dallas-mavericks" tabindex="0"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/dal.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"/></a><a class="AnchorLink" data-track-linkid="nba:schedule:team" data-track-navmethod="schedule" href="/nba/team/_/name/dal/dallas-mavericks" tabindex="0">Dallas</a></span>
-----
<span class="Table__Team"><a class="AnchorLink" data-track-linkid="nba:schedule:team" data-track-navmethod="schedule" href="/nba/team/_/name/wsh/washington-wizards" tabindex="0"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/wsh.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"/></a

<div class="ResponsiveTable"><div class="Table__Title">Wednesday, November 9, 2022 </div><div class="flex"><div class="Table__ScrollerWrapper relative overflow-hidden"><div class="Table__Shadow--left" style="opacity: 0;"></div><div class="Table__Scroller"><table class="Table" style="border-collapse: collapse; border-spacing: 0px;"><colgroup class="Table__Colgroup"><col class="Table__Column"><col class="Table__Column"><col class="Table__Column"><col class="Table__Column"><col class="Table__Column"></colgroup><thead class="Table__THEAD"><tr class="Table__TR Table__even"><th title="" colspan="2" class="Table__TH"><div class="">MATCHUP</div></th><th title="" class="Table__TH"><div class="">TIME</div></th><th title="" class="Table__TH"><div class="">TV</div></th><th title="" class="Table__TH"><div class="">tickets</div></th><th title="" class="Table__TH"><div class=""></div></th></tr></thead><tbody class="Table__TBODY"><tr class="Table__TR Table__TR--sm Table__even" data-idx="0"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/dal/dallas-mavericks"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/dal.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/dal/dallas-mavericks">DAL</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/orl/orlando-magic"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/orl.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/orl/orlando-magic">ORL</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468316">5:30 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/orlando-magic-tickets-amway-center-11-9-2022--sports-nba-basketball/production/4068141?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="1"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/por/portland-trail-blazers"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/por.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/por/portland-trail-blazers">POR</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/cha/charlotte-hornets"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/cha.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/cha/charlotte-hornets">CHA</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468314">7:00 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/charlotte-hornets-tickets-spectrum-center-11-9-2022--sports-nba-basketball/production/4068343?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="2"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/den/denver-nuggets"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/den.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/den/denver-nuggets">DEN</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/ind/indiana-pacers"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/ind.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/ind/indiana-pacers">IND</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468315">7:00 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/indiana-pacers-tickets-gainbridge-fieldhouse-11-9-2022--sports-nba-basketball/production/4068220?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="3"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/utah/utah-jazz"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/utah.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/utah/utah-jazz">UTAH</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/atl/atlanta-hawks"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/atl.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/atl/atlanta-hawks">ATL</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468317">7:30 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/atlanta-hawks-tickets-state-farm-arena-11-9-2022--sports-nba-basketball/production/4068242?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="4"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/det/detroit-pistons"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/det.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/det/detroit-pistons">DET</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/bos/boston-celtics"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/bos.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/bos/boston-celtics">BOS</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468318">7:30 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/boston-celtics-tickets-td-garden-11-9-2022--sports-nba-basketball/production/4067915?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="5"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/ny/new-york-knicks"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/ny.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/ny/new-york-knicks">NY</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/bkn/brooklyn-nets"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/bkn.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/bkn/brooklyn-nets">BKN</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468319">7:30 PM</a></td><td class="broadcast__col Table__TD"><div class="network-container"><a class="AnchorLink network-container_link" tabindex="0" href="/watch/"><figure class="Image aspect-ratio--parent Logo__Network network-espn"><div class="RatioFrame aspect-ratio--auto"></div><div class="Image__Wrapper Image__Wrapper--relative"><img alt="ESPN" class="" data-mptype="image" src="https://a.espncdn.com/redesign/assets/img/logos/networks/espn-red@2x.png"></div></figure></a></div></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/brooklyn-nets-tickets-barclays-center-11-9-2022--sports-nba-basketball/production/4068002?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="6"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/hou/houston-rockets"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/hou.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/hou/houston-rockets">HOU</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/tor/toronto-raptors"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/tor.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/tor/toronto-raptors">TOR</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468320">7:30 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/toronto-raptors-tickets-scotiabank-arena-11-9-2022--sports-nba-basketball/production/4068918?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="7"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/no/new-orleans-pelicans"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/no.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/no/new-orleans-pelicans">NO</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/chi/chicago-bulls"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/chi.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/chi/chicago-bulls">CHI</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468321">8:00 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/chicago-bulls-tickets-united-center-11-9-2022--sports-nba-basketball/production/4067947?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="8"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/phx/phoenix-suns"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/phx.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/phx/phoenix-suns">PHX</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/min/minnesota-timberwolves"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/min.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/min/minnesota-timberwolves">MIN</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468322">8:00 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/minnesota-timberwolves-tickets-target-center-11-9-2022--sports-nba-basketball/production/4068385?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="9"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/mil/milwaukee-bucks"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/mil.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/mil/milwaukee-bucks">MIL</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/okc/oklahoma-city-thunder"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/okc.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/okc/oklahoma-city-thunder">OKC</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468323">8:00 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/oklahoma-city-thunder-tickets-paycom-center-11-9-2022--sports-nba-basketball/production/4068658?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="10"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/mem/memphis-grizzlies"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/mem.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/mem/memphis-grizzlies">MEM</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/sa/san-antonio-spurs"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/sa.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/sa/san-antonio-spurs">SA</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468324">8:00 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/san-antonio-spurs-tickets-att-center-11-9-2022--sports-nba-basketball/production/4068197?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="11"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/lal/los-angeles-lakers"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/lal.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/lal/los-angeles-lakers">LAL</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/lac/la-clippers"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/lac.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/lac/la-clippers">LAC</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468325">10:00 PM</a></td><td class="broadcast__col Table__TD"><div class="network-container"><a class="AnchorLink network-container_link" tabindex="0" href="/watch/"><figure class="Image aspect-ratio--parent Logo__Network network-espn"><div class="RatioFrame aspect-ratio--auto"></div><div class="Image__Wrapper Image__Wrapper--relative"><img alt="ESPN" class="" data-mptype="image" src="https://a.espncdn.com/redesign/assets/img/logos/networks/espn-red@2x.png"></div></figure></a></div></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/la-clippers-tickets-cryptocom-arena-11-9-2022--sports-nba-basketball/production/4068322?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="12"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/cle/cleveland-cavaliers"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/cle.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/cle/cleveland-cavaliers">CLE</a></span></div></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">&nbsp; @ &nbsp;</span><span class="Table__Team"><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/sac/sacramento-kings"><img alt="" class="Image Logo v-mid team__logo Logo__sm" data-mptype="image" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/sac.png&amp;w=40&amp;h=40&amp;scale=crop&amp;cquality=40&amp;location=origin"></a><a class="AnchorLink" tabindex="0" data-track-navmethod="schedule" data-track-linkid="nba:schedule:team" href="/nba/team/_/name/sac/sacramento-kings">SAC</a></span></div></td><td class="date__col Table__TD"><a class="AnchorLink" tabindex="0" href="/nba/game?gameId=401468326">10:00 PM</a></td><td class="broadcast__col Table__TD"></td><td class="tickets__col Table__TD"><a class="AnchorLink TicketsColumn" tabindex="0" target="_blank" rel="noopener" href="https://www.vividseats.com/sacramento-kings-tickets-golden-1-center-11-9-2022--sports-nba-basketball/production/4068124?wsUser=717&amp;wsVar=us~nba~schedule,nba,en"><span>Buy Tickets<svg aria-hidden="true" class="external ml2 icon__svg" viewBox="0 0 24 24"><use xlink:href="#icon__external"></use></svg></span></a></td><td class="odds__col Table__TD"></td></tr></tbody></table></div><div class="Table__Shadow--right" style="opacity: 0;"></div></div></div></div>

In [46]:
chromedriver_autoinstaller.install()

options = Options()
options.headless = True


nba_schedule = 'https://www.espn.com/nba/schedule'
driver = webdriver.Chrome(options = options)
driver.get(nba_schedule)

nba_table = driver.find_elements_by_xpath ('//*[@id="fittPageContainer"]/div[3]/div/div/section/div/div[3]/div[1]/div/table/tbody/tr') #Prints first table from NFL schedule
#print(nba_table)
for itemFound in nba_table:
    print(itemFound.text)
driver.close()

In [ ]:
find_all_elements

In [76]:
import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.1.1" 
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
import pandas as pd
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro

In [78]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [79]:
# import R's "utils" package
utils = rpackages.importr('utils')

#install hoopR
utils.chooseCRANmirror(ind=1)
utils.install_packages("hoopR")
hoopR = rpackages.importr('hoopR')

NotImplementedError: Conversion 'rpy2py' not defined for objects of type '<class 'rpy2.rinterface.SexpClosure'>'

In [81]:
#install hoopR
utils.chooseCRANmirror(ind=1)
utils.install_packages("hoopR")
hoopR = rpackages.importr('hoopR')

NameError: name 'utils' is not defined

SyntaxError: invalid syntax (<ipython-input-80-cf1043e4b35f>, line 2)